In [1]:
#Imports
import foursquare
import pandas as pd
import numpy as np
from scripts.constants import *
from scripts import dataframe_builder, api_request

In [6]:
# Collect new random samples
SAMPLES_TO_COLLECT = 0
api_request.collect_random_samples(SAMPLES_TO_COLLECT)

In [4]:
# Reconcile dataset with logged API responses
dataframe_builder.update_dataframe()

In [3]:
# Make requests for missing information
api_request.request_missing_information()

./data/response_logs/foursquare_venue_search.json Request Failed: 402
{"meta":{"code":402,"errorType":"credits_exhausted","errorDetail":"Credits exhausted","requestId":"61f97e4f97d7ca08b7db9d6e"},"response":{}}
./data/response_logs/foursquare_venue_search.json Request Failed: 402
{"meta":{"code":402,"errorType":"credits_exhausted","errorDetail":"Credits exhausted","requestId":"61f97e4f97c17f06cc03d235"},"response":{}}
./data/response_logs/foursquare_venue_search.json Request Failed: 402
{"meta":{"code":402,"errorType":"credits_exhausted","errorDetail":"Credits exhausted","requestId":"61f97e4fd19f121d0488d12b"},"response":{}}
./data/response_logs/foursquare_venue_search.json Request Failed: 402
{"meta":{"code":402,"errorType":"credits_exhausted","errorDetail":"Credits exhausted","requestId":"61f97e509eb5461dadf6cad6"},"response":{}}
./data/response_logs/foursquare_venue_search.json Request Failed: 402
{"meta":{"code":402,"errorType":"credits_exhausted","errorDetail":"Credits exhausted",

In [7]:
df = pd.read_csv(SAVE_DIR+RESULTS, index_col=0)
df

,country,city,yelp,four_square,google
"55.24601, -107.58192",Canada,La Plonge 192,0.0,1.0,0.0
"27.7847, 61.30295",Iran,Qaderabad,0.0,0.0,3.0
"53.16644, -126.31993",Canada,Ice Hill,0.0,0.0,0.0
"-17.38333, -48.4",Brazil,Palmelo,0.0,30.0,5.0
"58.32333, 100.09528",Russia,Kosoy Byk,0.0,0.0,1.0
"52.41653, 133.33867",Russia,Taloma,0.0,0.0,0.0
"-4.51811, 151.62302",Papua New Guinea,Poiniara,0.0,0.0,1.0
"46.90654, -67.38997",Canada,Plaster Rock,1.0,20.0,18.0
"66.88996, 18.40836",Sweden,Luohkáluokta,0.0,1.0,2.0
"53.16605, 139.05147",Russia,Kaygachan,0.0,0.0,0.0


In [9]:
### Compare Coverage
fs = df[['country','four_square']].groupby('country').sum()
fs_countries = len(fs[fs['four_square']!= 0])
fs_venues = sum(fs['four_square'])

yelp = df[['country','yelp']].groupby('country').sum()
yelp_countries = len(yelp[yelp['yelp']!= 0])
yelp_venues = sum(yelp['yelp'])

google = df[['country','google']].groupby('country').sum()
google_countries = len(google[google['google']!= 0])
google_venues = sum(google['google'])

samples = len(df)
print(f''' 
      From {samples} samples:''')

print(f''' 
      Four Square:
      {fs_venues} venues across {fs_countries} countries
      ''')
print(f''' 
      Google:
      {google_venues} venues across {google_countries} countries
      ''')
print(f''' 
      Yelp:
      {yelp_venues} venues across {yelp_countries} countries
      ''')

 
      From 25 samples:
 
      Four Square:
      115.0 venues across 7 countries
      
 
      Google:
      135.0 venues across 15 countries
      
 
      Yelp:
      51.0 venues across 4 countries
      
